# Importing images

Microscopy images come in various formats, some "general" like tiff and some proprietary like microscope manufacturer formats like czi, nd2, lif etc. The problem is that data and metadata are encoded in a different way in each of those formats. Therefore for each one, we need a piece of software capable of reading it. There are multiple solutions in Python to do that and we'll explore a series of examples here.

## scikit-image

scikit-image provides a simple importer via its ```io``` submodule. It handles most "simple" standard formats. In this course you will be able to import most images with:

In [1]:
import skimage

In [2]:
image = skimage.io.imread('../data/idr9036378_cropped.tif')

In [3]:
image.shape

(14, 8, 2, 262, 290)

This provides a simple Numpy array as output. Note that with this approach any metadata regarding pixel size, scale, dimensions etc. is lost.

Note also that you can provide either a local path to your image (absolute or relative) or a url. For example this works too:

In [4]:
image = skimage.io.imread('https://zenodo.org/record/4485316/files/Flue-N2DH-GOWT1_02_GT.tif?download=1')

In [5]:
image.shape

(1024, 1024)

## More formats 

If you want to recover more information about the images as well as be able to open a larger variety of formats, we recommend the [```aicsimageio```](https://allencellmodeling.github.io/aicsimageio/) library maintained by the All Institute for Cell Sciences. The mechanisms here is a bit different: we define an *object* by specifying the image location and then we request information from that object. For example we can open a czi file:

In [6]:
from aicsimageio import AICSImage

In [7]:
myfile = AICSImage('../data/C_Ss_3D-LR40x.czi')

Now we can extract information about the dimensions:

In [8]:
myfile.dims

<Dimensions [T: 1, C: 3, Z: 52, Y: 512, X: 512]>

We see that we have a 512x512 image with 52 planes and 3 channels. Naturally we can also access the actual pixel values in the form of an array:

In [9]:
mypixels = myfile.data

In [10]:
mypixels.shape

(1, 3, 52, 512, 512)

Note that the library offers some more advanced data formats (dask arrays) that allow one to perform computations on image blocks in case we are dealing with very large images. We will learn more about this when exploring the Dask library.

## Specialized libraries

Both scikit-image and aicsimageio take care of common light microscopy formats. However you might deal with more exotic formats not covered by those libraries. For example formats from electron microscopy like mrc, or medical data like DICOM. In those cases, you can almost always find specific libraries dedicated to those formats. 

### DICOM

For DICOM you can for example use the [pydicom](https://pydicom.github.io/pydicom/stable/) library:

In [12]:
pip install pydicom

  Using cached pydicom-2.3.0-py3-none-any.whl (2.0 MB)
Note: you may need to restart the kernel to use updated packages.


As an example, we use here a dataset from Zenodo, which is an MRI scan of a head: https://zenodo.org/record/16956.

Using the pydicom package we can now import a single slice using the ```dcmread()``` function:

In [13]:
import pydicom

In [14]:
single_slice = pydicom.dcmread('../data/DICOM/ST000000/SE000000/MR000000')

The ```single_slice``` variable is an object that doesn't only contain pixel values but also all metadata from the DICOM file. For example we can get information about the patient or the pixel size:

In [15]:
single_slice.PatientName

'LIONHEART^WILLIAM'

In [16]:
single_slice.PixelSpacing

[0.9765625, 0.9765625]

But of course we can also access the raw pixel values as a Numpy array:

In [17]:
single_slice.pixel_array

array([[ 8, 12, 10, ...,  5,  9,  5],
       [ 4, 13, 16, ...,  2,  7,  5],
       [13, 18, 11, ...,  8, 10,  6],
       ...,
       [10, 10,  7, ..., 24, 17,  3],
       [16, 13,  7, ..., 10, 14, 16],
       [11,  9, 11, ...,  6,  8, 13]], dtype=uint16)

### MRC

Similarly to read files in .mrc format, which is widely used in EM we are using the [mrcfile](https://github.com/ccpem/mrcfile) package. Again, we first need to install it:

In [18]:
pip install mrcfile

  Using cached mrcfile-1.4.0-py2.py3-none-any.whl (42 kB)
Note: you may need to restart the kernel to use updated packages.


In [19]:
import mrcfile

Now we can open the file and then access its contents:

In [20]:
mrc =  mrcfile.open('../data/10392_S9_File_PVM_Rupture.mrc')

In [21]:
mrc.data.shape

(190, 617, 617)

As we have opened the image and not directly imported it, note that we need to close the image. This is particularly important in case we open many images that would then take a lot of space in memory: 

In [22]:
mrc.close()

## Exercise

1. Import one of the tif files that you can find in this Zenodo repository using scikit-image: https://zenodo.org/record/4266199#.YsKgKuzP2rM

2. Import the file ```20180209_mitoBFP_Drp1GFP_KDEL-RFP.lif``` using aicsimageio. What are the channel names? What are the dimensions? Access the data as a Numpy array and check that the dimensions match.